# Mini-Project: Building a GNN-Based Recommender System

**Course:** Recommender Systems

**Estimated Time:** 4 Hours

**Topic:** Link Prediction for Music Recommendations

### 1. Project Goal

The objective of this project is to build a recommender system that suggests new music artists to users. You will model the user-artist relationship as a **bipartite graph** and frame the recommendation task as a **link prediction problem**. You will implement a Graph Neural Network (GNN) to learn embeddings for users and artists, which will then be used to predict potential future interactions (i.e., which artists a user might like).

### 2. Core Technologies

You will use the following key libraries:

* **pandas:** For data loading and initial manipulation.

* **NetworkX:** To construct the initial user-artist graph.

* **PyTorch & PyTorch Geometric (PyG):** For defining the GNN model, preparing the data, and training.

* **scikit-learn:** For evaluating the model's performance.

### 3. The Dataset: Last.fm User-Artist Interactions

We will use a dataset from the music streaming service **Last.fm**. It contains a list of users, the artists they have listened to, and the number of times they've played songs by that artist (a "weight"). This represents a rich set of implicit feedback.

You can download the dataset here: [Last.fm Asia Social Network Dataset](https://files.grouplens.org/datasets/hetrec2011/hetrec2011-lastfm-2k.zip)

The key file you'll need is `user_artists.dat`.

### 4. Step-by-Step Implementation Guide

---

#### **Step 1: Environment Setup & Data Loading (Est. 20 mins)**

Open the PowerShell Prompt as admin (in Windows) or the Terminal (in Mac and Linux), and strat by creating a conda environment:

`conda create --name rec_gnn python=3.9`

Install the demanded libraries:

`pip install torch torch-geometric pandas networkx scikit-learn`

Finally choose the kernel `rec_gnn` on VsCode to execute your code.

**Load Data:** Use the `pandas` library to load the `user_artists.dat` file.
  * Use the function `pd.read_csv()`, specifying the file path and setting the separator to `sep='\t'`.
  * Assign the column names: `['userID', 'artistID', 'listen_count']`.

**Explore:** Print the head of the DataFrame and the number of unique users and artists to understand the data's scale.

In [222]:
# All imports for the project
import pandas as pd
import networkx as nx
import torch
import torch.nn as nn
import torch.optim as optim
from torch_geometric.data import Data
from torch_geometric.nn import SAGEConv
from torch_geometric.utils import negative_sampling
import torch_geometric.transforms as T
from sklearn.metrics import roc_auc_score
import torch.nn.functional as F

# Your code starts here

In [223]:
df=pd.read_csv("user_artists.dat", sep='\t')

In [224]:
df = df.rename(columns={"weight": "listen_count"})

In [225]:
df.head(10)

,userID,artistID,listen_count
0,2,51,13883
1,2,52,11690
2,2,53,11351
3,2,54,10300
4,2,55,8983
5,2,56,6152
6,2,57,5955
7,2,58,4616
8,2,59,4337
9,2,60,4147


#### **Step 2: Building the Bipartite Graph with NetworkX (Est. 30 mins)**

Now, convert the DataFrame of interactions into a single, undirected bipartite graph. A bipartite graph has two distinct sets of nodes, and edges only connect nodes from different sets (in our case, users to artists).

1.  **Instantiate a Graph:** Create an empty graph object using `G = nx.Graph()`.
2.  **Add Nodes with Attributes:** Iterate through the unique users and artists. For each one, you must add it to the graph with a special attribute to identify its type.
    * For users, use `G.add_node(user_id, bipartite=0)`.
    * For artists, use `G.add_node(artist_id, bipartite=1)`.
3.  **Add Edges:** Iterate through each row of your DataFrame. For each row, create a connection between the user and the artist using `G.add_edge(row['userID'], row['artistID'])`. We will ignore the `listen_count` for this model.

In [226]:
G = nx.Graph()

In [227]:
for user_id in df["userID"].unique():
    G.add_node(user_id, bipartite=0)

for artist_id in df["artistID"].unique():
    G.add_node(artist_id, bipartite=1)

In [228]:
for row in df.itertuples(index=False):
    G.add_edge(int(row.userID), int(row.artistID))


#### **Step 3: Preparing the Data with PyTorch Geometric (Est. 40 mins)**

Next, you'll convert the NetworkX graph into a format that PyTorch Geometric can use. This involves creating a single `Data` object and then splitting its edges for training, validation, and testing.

1.  **Map Node IDs:** Create a dictionary that maps every original `userID` and `artistID` to a unique integer from 0 to `num_nodes - 1`. This is required for the embedding layer.
2.  **Create PyG Data Object:** Construct a `torch_geometric.data.Data` object. This will hold all the information about our graph.
    * **Node Features (`x`):** We don't have initial features, so we'll use learnable embeddings instead. Create a tensor of node indices that the model will use to look up these embeddings. Use `torch.arange(num_nodes, dtype=torch.long)`.
    * **Edge Index (`edge_index`):** Convert the list of graph edges into a tensor of shape `[2, num_edges]` where each column is an edge. Set its `dtype` to `torch.long`.
3.  **Split Edges:** Use the modern `torch_geometric.transforms.RandomLinkSplit` transform to partition the edges. This is the standard method for link prediction tasks.
    * Instantiate the transform: `transform = T.RandomLinkSplit(...)`.
    * Inside the transform, set key parameters:
        * `num_val=0.1` and `num_test=0.1` to hold out 10% of edges for validation and 10% for testing.
        * `is_undirected=True` because our graph is undirected.
        * `add_negative_train_samples=False` because we will perform negative sampling inside the training loop.
        * `split_labels=True` to create the necessary `pos_edge_label_index` attributes.
    * Apply it to your `data` object to get three new objects: `train_data, val_data, test_data = transform(data)`.

In [229]:
all_ids = pd.concat([df['userID'], df['artistID']]).unique()
id_mapping = {original_id: new_id for new_id, original_id in enumerate(all_ids)}

In [230]:
len (all_ids)

17644

In [231]:
X = torch.arange(len(id_mapping), dtype=torch.long)

In [232]:
df_map = df[["userID","artistID"]].copy()
df_map["userID"]= df_map["userID"].map(id_mapping)
df_map["artistID"] = df_map["artistID"].map(id_mapping)

In [233]:
df_map.head(10)

,userID,artistID
0,0,48
1,0,49
2,0,50
3,0,51
4,0,52
5,0,53
6,0,54
7,0,55
8,0,56
9,0,57


In [234]:
edge_index = torch.tensor(
    df_map[['userID', 'artistID']].values.T,  # transpose pour avoir [2, num_edges]
    dtype=torch.long
)

In [235]:
edge_index

tensor([[    0,     0,     0,  ...,  1891,  1891,  1891],
        [   48,    49,    50,  ..., 17641, 17642, 17643]])

In [236]:
data = Data(x=X, edge_index=edge_index)


In [237]:
from torch_geometric.transforms import RandomLinkSplit

transform = RandomLinkSplit(
    num_val=0.1,                 
    num_test=0.1,               
    is_undirected=True,  
    add_negative_train_samples=False,
    split_labels=True
)

In [238]:
train_data, val_data, test_data = transform(data)

#### **Step 4: Implementing the GNN Model (Est. 60 mins)**

Define your GNN using PyTorch and PyG. The model will learn dense vector representations (embeddings) for all users and artists.

1.  **Create the Class:** Define a class `GNNLinkPredictor` that inherits from `torch.nn.Module`.
2.  **Define Layers in `__init__`:** The constructor should accept `num_nodes`, `embedding_dim`, `hidden_channels`, and `out_channels` as arguments.
    * **Embedding Layer:** Create an `nn.Embedding(num_nodes, embedding_dim)` layer. This will store the learnable feature vector for every node in the graph.
    * **Convolution Layers:** Define two `SAGEConv` layers. The first layer's `in_channels` must match your `embedding_dim`.
3.  **Implement the Encoder:** Create an `encode` method that takes node indices (`x`) and the message-passing edges (`edge_index`) as input.
    * First, pass the node indices `x` through your `self.embedding` layer to get the dense node features.
    * Then, process these features through the two `SAGEConv` layers, using a ReLU activation function in between.
4.  **Implement the Decoder:** Create a `decode` method that takes the final node embeddings (`z`) and a set of query edges (`edge_label_index`) as input.
    * Use the `edge_label_index` to select the embeddings for the source and destination nodes of each edge.
    * Calculate the dot product between the source and destination node embeddings for each edge. This dot product will be our link prediction score.

In [239]:

class GNNLinkPredictor(nn.Module):
    
    def __init__(self, num_nodes, embedding_dim, hidden_channels, out_channels):
        super().__init__()

        self.embeding=nn.Embedding(num_nodes, embedding_dim)
        
        self.conv1=SAGEConv(embedding_dim,hidden_channels)
        self.conv2=SAGEConv(hidden_channels,out_channels)
        
        self.relu = nn.ReLU()
    
    def encoder(self,x,edge_index):
        x=self.embeding(x)

        x = self.conv1(x, edge_index)
        x = self.relu(x)
        x = self.conv2(x, edge_index)

        return x

    def decoder(self,z,edge_index_label):
        source = z[edge_index_label[0]]
        destination = z[edge_index_label[1]]

        product = (source * destination).sum(dim=1)

        return product

    def forward(self,x, edge_index,edge_label_index):
        z=self.encoder(x,edge_index)
        return self.decoder(z,edge_label_index)
        
        
        

#### **Step 5: Training the Model (Est. 60 mins)**

With the model and data ready, you will write a standard PyTorch training loop to optimize the model's parameters.

1.  **Initialize:** Instantiate your `GNNLinkPredictor` model, an `Adam` optimizer, and the `BCEWithLogitsLoss` loss function, which is ideal for binary classification tasks like link prediction.
2.  **Define the `train` function:** This function will perform one full training step. It should accept the `train_data` object as an argument.
3.  **Inside the function:**
    * Get the node indices (`x`), message-passing edges (`edge_index`), and positive supervision links (`pos_edge_label_index`) from the `train_data` object.
    * **Negative Sampling:** For every positive link, we need a negative one. Use the `torch_geometric.utils.negative_sampling` function to generate a tensor of non-existent edges. Set `num_neg_samples` to be equal to the number of positive edges.
    * Combine the positive and negative supervision links into a single `edge_label_index` tensor.
    * Pass the node indices, message-passing edges, and combined supervision links to the model to get predictions.
    * Create the ground-truth `edge_label` tensor: a tensor of 1s for the positive links and 0s for the negative links.
    * Calculate the loss, backpropagate, and update the model weights with `optimizer.step()`.
4.  **Run the Loop:** Call your `train(train_data)` function repeatedly in a `for` loop for a set number of epochs.

In [240]:
model=GNNLinkPredictor(len(all_ids),20,10,1)
optimizer=torch.optim.Adam(model.parameters(),lr=0.1)

In [241]:
def train (train_data):
    edge_index = train_data.edge_index
    pos_edge_label_index=train_data.pos_edge_label_index
    dimention=len(pos_edge_label_index[1])
    neg_sampling= negative_sampling(edge_index=edge_index,num_neg_samples=dimention)
    edge_label_index=  torch.cat([pos_edge_label_index, neg_sampling], dim=1)

    model.train()
    optimizer.zero_grad()
    edge_label=torch.cat([torch.ones(dimention),torch.zeros(dimention)])
    pred = model(train_data.x, train_data.edge_index, edge_label_index)



    loss=F.binary_cross_entropy_with_logits(pred,edge_label)
    loss.backward()
    optimizer.step()
    return loss.item()

In [242]:
num_epochs = 100
losses = []

for epoch in range(1, num_epochs + 1):
    loss = train(train_data)
    losses.append(loss)
    
    if epoch % 5 == 0:
        print(f"Epoch {epoch}/{num_epochs}, Loss: {loss:.4f}")

Epoch 5/100, Loss: 0.7128
Epoch 10/100, Loss: 0.6918
Epoch 15/100, Loss: 0.6803
Epoch 20/100, Loss: 0.6177
Epoch 25/100, Loss: 0.5336
Epoch 30/100, Loss: 0.5206
Epoch 35/100, Loss: 0.5063
Epoch 40/100, Loss: 0.5040
Epoch 45/100, Loss: 0.5013
Epoch 50/100, Loss: 0.4988
Epoch 55/100, Loss: 0.4986
Epoch 60/100, Loss: 0.4943
Epoch 65/100, Loss: 0.4909
Epoch 70/100, Loss: 0.4933
Epoch 75/100, Loss: 0.4934
Epoch 80/100, Loss: 0.4928
Epoch 85/100, Loss: 0.4918
Epoch 90/100, Loss: 0.4929
Epoch 95/100, Loss: 0.4906
Epoch 100/100, Loss: 0.4921


#### **Step 6: Evaluation (Est. 20 mins)**

Finally, evaluate your trained model's ability to distinguish between true and false links on the unseen test dataset. We will use the **AUC (Area Under the ROC Curve)** metric.

1.  **Define the `test` function:** Create a function that accepts a data object (e.g., `val_data` or `test_data`). It should be decorated with `@torch.no_grad()` to disable gradient calculations.
2.  **Inside the function:**
    * Set the model to evaluation mode with `model.eval()`.
    * Get the necessary tensors from the input `data` object: `x`, `edge_index` (for encoding), `pos_edge_label_index`, and `neg_edge_label_index` (for decoding).
    * First, generate the final node embeddings by calling `model.encode(x, edge_index)`.
    * Then, get the prediction scores for the test links by calling `model.decode(z, edge_label_index)`.
    * Create the ground-truth labels (1s and 0s) for the positive and negative test links.
    * Use `sklearn.metrics.roc_auc_score` to compute the AUC score between your predictions and the true labels.
3.  **Run Evaluation:** Call your `test` function with the `test_data` object to get the final performance score.

In [243]:
# Your code here
from sklearn.metrics import roc_auc_score

@torch.no_grad()
def test(test_data):
    model.eval()
    
    x=test_data.x
    edge_index = test_data.edge_index
    z = model.encoder(x, edge_index)
    
    pos_edge_label_index=test_data.pos_edge_label_index
    neg_edge_label_index=test_data.neg_edge_label_index
    edge_label_index = torch.cat([pos_edge_label_index, neg_edge_label_index], dim=1)
    
    pred=model.decoder(z, edge_label_index)

    pos_labels = torch.ones(pos_edge_label_index.size(1))
    neg_labels = torch.zeros(neg_edge_label_index.size(1))
    edge_labels = torch.cat([pos_labels, neg_labels])

    auc = roc_auc_score(edge_labels.cpu().numpy(), pred.cpu().numpy())
    return auc

In [244]:
test_auc = test(test_data)
print(f"Test AUC: {test_auc:.4f}")

Test AUC: 0.7830


### 7. Final Deliverables

* A Python script (`.ipynb`) containing the full implementation of all steps.

* A brief summary (in comments or a markdown cell) reporting your final test AUC score and a short reflection:

    * Did the model perform better than random guessing?

    * What is one way you could potentially improve the model if you had more time?

### Final result

Final AUC = 0.7830

The model perform better than random guessing because it has 0.7830 of AUC that is better than 0.5 

To improve the result we can expand the model to have a deeper model. We can also try to tune the hyperparameters or have more data.

